# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [17]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

In [2]:
from random import randrange

In [3]:
import time
import numpy as np

In [19]:
def adj_list(v, e):
    n = len(v)
    E = [[] for i in range(n + 1)]
    for v, u in e:
        E[v].append(u)
        E[u].append(v)
    for i in range(n + 1):
        E[i] = set(E[i])
    return E

def adj_matr(v, e):
    n = len(v)
    M = [[False] * (n + 1) for i in range(n + 1)]
    for v, u in e:
        M[v][u] = True
        M[u][v] = True
    return M

def edges_between(V1, V2):
    res = 0
    for x in V1:
        res += len(E[x] & V2)
    return res

def find_best_swap(V1, V2, maxN, used):
    max_delta = -len(V1) ** 2 - len(V2) ** 2
    argmin = (0, 0)
    for i in range(maxN):
        v = list(V1)[randrange(len(V1))]
        u = list(V2)[randrange(len(V2))]
        if used[v] or used[u]:
            continue
        V1_P = V1 - {v}
        V2_P = V2 - {u}
        delta = edges_between({u}, V1_P) + edges_between({v}, V2_P) - \
                edges_between({v}, V1_P) - edges_between({u}, V2_P)
        if delta > max_delta:
            max_delta = delta
            argmin = (v, u)
    return (max_delta, argmin)

def from_canon_by_sparrows(V1, V2, used):
    n = len(V1) + len(V2)
    diff_exch = [0] * (n + 1)
    for u, v in e:
        if (u in V1) != (v in V1):
            diff_exch[u] -= 1
            diff_exch[v] -= 1
        else:
            diff_exch[u] += 1
            diff_exch[v] += 1
    diff_exch = np.array(diff_exch)
    set_of_used = set()
    for i in range(n + 1):
        if used[i]:
            set_of_used.add(i)
    L1 = list(V1 - set_of_used)
    L2 = list(V2 - set_of_used)
    de1 = diff_exch[L1]
    de2 = diff_exch[L2]
    cut_M = (np.array(M))[np.ix_(L1, L2)]
    old = edges_between(V1, V2)
    new_V1 = (V1 - {L1[1]}) | {L2[0]}
    new_V2 = (V2 - {L2[0]}) | {L1[1]}
    new = edges_between(new_V1, new_V2)
    De1 = np.tile(de1, de2.shape[0]).reshape((de2.shape[0], de1.shape[0]))
    De2 = np.tile(de2, de1.shape[0]).reshape((de1.shape[0], de2.shape[0])).T
    F = (De1 + De2) // 2 + cut_M.T
    best_res = np.min(F)
    p = np.argmin(F)
    j = p // cut_M.shape[0]
    i = p % cut_M.shape[0]
    v = L1[i]
    u = L2[j]
    return (-best_res, (v, u))
    

def variable_depth_local_search(graph, hard_alg = True):
    time_start = time.clock()
    n = len(graph[0])
    V = list(graph[0])
    V1 = set(V[:n // 2])
    V2 = set(graph[0]) - V1
    cur_f = edges_between(V1, V2)
    print "starting point:", cur_f, "edges"
    n_iter = 0
    iterKL = 0
    while True:
        if time.clock() - time_start > 60:
            break
        n_iter += 1
        if n_iter % report_freq == 0:
            print "current result:", cur_f
        used = [False] * (n + 1)
        delta, argmin = find_best_swap(V1, V2, maxLS, used)
        v = argmin[0]
        u = argmin[1]
        V1 = (V1 - {v}) | {u}
        V2 = (V2 - {u}) | {v}
        cur_f -= delta
        if delta <= 0:
            old_V1 = V1
            old_V2 = V2
            sum_delta = delta
            V1 = (V1 - {v}) | {u}
            V2 = (V2 - {u}) | {v}
            used[v] = True
            used[u] = True
            for i in range(max_depth - 1):
                iterKL += 1
                if iterKL % report_freq == 0:
                    print 'after KL:', cur_f
                delta, argmin = find_best_swap(V1, V2, maxKL, used)
                cur_f -= delta
                sum_delta += delta
                v = argmin[0]
                u = argmin[1]
                V1 = (V1 - {v}) | {u}
                V2 = (V2 - {u}) | {v}
                if time.clock() - time_start > 60:
                    if sum_delta <= 0:
                        V1 = old_V1
                        V2 = old_V2
                    break
            if sum_delta <= 0:
                sum_delta1 = sum_delta
                if hard_alg:
                    used = [False] * (n + 1)
                    for i in range(max_depth_f):
                        if time.clock() - time_start > 60:
                            if sum_delta1 <= 0:
                                V1 = old_V1
                                V2 = old_V2
                            return V1
                        delta, (v, u) = from_canon_by_sparrows(V1, V2, used)
                        used[u] = True
                        used[v] = True
                        cur_f -= delta
                        print "after FCBS:", cur_f
                        sum_delta1 += delta
                        V1 = (V1 - {v}) | {u}
                        V2 = (V2 - {u}) | {v}
                if sum_delta1 <= 0:
                    V1 = old_V1
                    V2 = old_V2
                break
    return V1

In [13]:
%%time
v, e = read_instance('add20.graph')
E = adj_list(v, e)
M = adj_matr(v, e)
max_depth = 10
max_depth_f = 10
maxLS = 300
maxKL = 500
report_freq = 20
V1 = variable_depth_local_search((v, e))
V2 = set(v) - V1
print(edges_between(V1, V2))

('starting point:', 1927, 'edges')
('current result:', 1858)
('current result:', 1776)
('current result:', 1715)
('current result:', 1664)
('current result:', 1610)
('current result:', 1565)
('current result:', 1529)
('current result:', 1487)
('current result:', 1455)
('current result:', 1405)
('current result:', 1352)
('after KL:', 1323)
('current result:', 1302)
('after KL:', 1256)
('current result:', 1236)
('after KL:', 1203)
('current result:', 1183)
('after KL:', 1176)
('after KL:', 1153)
('current result:', 1131)
('after KL:', 1130)
('after KL:', 1106)
('after KL:', 1091)
('after KL:', 1082)
('after KL:', 1070)
('current result:', 1059)
('after KL:', 1058)
('after KL:', 1051)
('after FCBS:', 1049)
('after FCBS:', 1047)
('after FCBS:', 1046)
('after FCBS:', 1045)
('after FCBS:', 1044)
('after FCBS:', 1042)
('after FCBS:', 1041)
('after FCBS:', 1040)
('after FCBS:', 1039)
('after FCBS:', 1038)
1038
CPU times: user 24.6 s, sys: 788 ms, total: 25.3 s
Wall time: 23.8 s


In [14]:
%%time
v, e = read_instance('cti.graph')
E = adj_list(v, e)
M = adj_matr(v, e)
print(len(v), len(e))
max_depth = 3
max_depth_f = 10
maxLS = 1000
maxKL = 100
report_freq = 1
V1 = variable_depth_local_search((v, e))
V2 = set(v) - V1
print(edges_between(V1, V2))

(16840, 96464)
('starting point:', 1206, 'edges')
('current result:', 1206)
('after KL:', 1209)
('after KL:', 1215)
('after FCBS:', 1213)
('after FCBS:', 1207)
('after FCBS:', 1205)
('after FCBS:', 1203)
('after FCBS:', 1199)
('after FCBS:', 1197)
('after FCBS:', 1193)
('after FCBS:', 1192)
('after FCBS:', 1193)
('after FCBS:', 1193)
1193
CPU times: user 59.6 s, sys: 6.51 s, total: 1min 6s
Wall time: 1min 6s


In [24]:
%%time
v, e = read_instance('t60k.graph')
E = adj_list(v, e)
#M = adj_matr(v, e)
print(len(v), len(e))
max_depth = 100
max_depth_f = 10
maxLS = 1000
maxKL = 1000
report_freq = 1
V1 = variable_depth_local_search((v, e), False)
V2 = set(v) - V1
print(edges_between(V1, V2))

(60005, 178880)
starting point: 322 edges
current result: 322
after KL: 324
after KL: 328
after KL: 332
after KL: 335
after KL: 339
after KL: 342
after KL: 343
after KL: 343
after KL: 345
after KL: 347
after KL: 347
after KL: 350
after KL: 353
after KL: 355
after KL: 357
after KL: 357
after KL: 359
after KL: 360
after KL: 363
after KL: 366
after KL: 366
after KL: 368
after KL: 368
after KL: 370
after KL: 374
after KL: 377
after KL: 379
after KL: 380
after KL: 380
after KL: 380
after KL: 382
after KL: 382
after KL: 384
324
CPU times: user 1min 2s, sys: 196 ms, total: 1min 2s
Wall time: 1min 1s


In [22]:
%%time
v, e = read_instance('m14b.graph')
E = adj_list(v, e)
#M = adj_matr(v, e)
print(len(v), len(e))
max_depth = 10
max_depth_f = 10
maxLS = 70
maxKL = 100
report_freq = 1
V1 = variable_depth_local_search((v, e), False)
V2 = set(v) - V1
print(edges_between(V1, V2))

(214765, 3358036)
starting point: 234531 edges
current result: 234531
after KL: 234543
after KL: 234537
after KL: 234526
after KL: 234522
after KL: 234513
after KL: 234513
after KL: 234512
after KL: 234501
after KL: 234497
current result: 234495
after KL: 234505
after KL: 234508
after KL: 234509
after KL: 234505
after KL: 234495
after KL: 234485
after KL: 234476
after KL: 234473
after KL: 234463
current result: 234454
after KL: 234459
after KL: 234454
after KL: 234453
after KL: 234446
after KL: 234446
after KL: 234442
after KL: 234431
after KL: 234435
after KL: 234431
current result: 234420
after KL: 234425
after KL: 234410
after KL: 234406
after KL: 234400
after KL: 234400
after KL: 234400
after KL: 234401
after KL: 234401
after KL: 234397
current result: 234396
after KL: 234408
after KL: 234390
after KL: 234389
after KL: 234381
after KL: 234367
after KL: 234364
after KL: 234362
after KL: 234356
after KL: 234353
current result: 234353
after KL: 234361
after KL: 234354
after KL: 234352

In [29]:
import time

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph', 'm14b.graph']
    for filename in filenames:
        instance = read_instance(filename)
        print('Solving instance {}…'.format(filename), end='')
        time_start = time.clock()
        quality = get_quality(instance, variable_depth_local_search(instance))
        time_elapsed = time.clock()-time_start
        print(' done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

In [30]:
run_all()

Solving instance add20.graph… done in 0.015 seconds with quality 11070
Solving instance cti.graph… done in 0.16 seconds with quality 94052
Solving instance t60k.graph… done in 0.23 seconds with quality 178236
Solving instance m14b.graph… done in 5.4 seconds with quality 2888974


## Выводы
Локальный поиск с эвристикой Кернигана-Лина находит небольшое улучшение почти во всех случаях. Небольшое, так как, наверное, во входных данных starting point - точка, близкая к локальному минимуму, и выкарабкиваться из него долго. Нормального отсечения перебора я не придумал, поэтому вместо всех пар (вершина слева, вершина справа) выбирал несколько (maxLS) случайных пар. В алгоритме Кернигана-Лина я выбирал maxKL случайных пар, находил наилучшую среди них, и это делал max_depth шагов.
Параметры для тестов (maxLS, maxKL, max_depth):
add20 - 300, 500, 10
cti - 1000, 100, 3
t60k - 1000, 1000, 100 (и всё равно из локального минимума не выкарабкался)
m14b - 70, 100, 10